<a href="https://colab.research.google.com/github/jozeni00/csci166_deep_q_learning/blob/main/c166f25_02b_dqn_pong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 3 --- Final Test for 06.11.2025

In [16]:
!pip install gymnasium[atari,accept-rom-license]
!pip install autorom
!pip install stable-baselines3

In [17]:
!AutoROM --accept-license

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.12/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.


# Install the Gym

In [18]:
import ale_py
import gymnasium as gym

# Configure the model save drive

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import os
save_dir = "/content/drive/MyDrive/PUBLIC/Models"
os.makedirs(save_dir, exist_ok=True)

# SETTINGS

In [21]:
game1_ale = "ALE/Tetris-v5"
game2_ale = "ALE/Frogger-v5"

# ----- GAME 1 -----

# GAME 1 Model

In [22]:
from dataclasses import dataclass
import argparse
import time
from datetime import datetime
import numpy as np
from moviepy.editor import ImageSequenceClip
import collections
import typing as tt


import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard.writer import SummaryWriter

In [23]:
#dqn_model
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        size = self.conv(torch.zeros(1, *input_shape)).size()[-1]
        self.fc = nn.Sequential(
            nn.Linear(size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    def forward(self, x: torch.ByteTensor):
        x = x.float() / 255.0
        return self.fc(self.conv(x))

In [24]:
#wrappers

from gymnasium import spaces
from stable_baselines3.common import atari_wrappers


class ImageToPyTorch(gym.ObservationWrapper):
    """
    ImageToPyTorch: Reorders image dimensions from (H, W, C) to (C, H, W)
    for compatibility with PyTorch convolutional layers.
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        obs = self.observation_space
        assert isinstance(obs, gym.spaces.Box)
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = gym.spaces.Box(
            low=obs.low.min(), high=obs.high.max(),
            shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    BufferWrapper: Maintains a rolling window of the last `n_steps` frames
    to give the agent a sense of temporal context.
    """
    def __init__(self, env, n_steps):
        super(BufferWrapper, self).__init__(env)
        obs = env.observation_space
        assert isinstance(obs, spaces.Box)
        new_obs = gym.spaces.Box(
            obs.low.repeat(n_steps, axis=0), obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype)
        self.observation_space = new_obs
        self.buffer = collections.deque(maxlen=n_steps)

    def reset(self, *, seed: tt.Optional[int] = None, options: tt.Optional[dict[str, tt.Any]] = None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset()
        return self.observation(obs), extra

    def observation(self, observation: np.ndarray) -> np.ndarray:
        self.buffer.append(observation)
        return np.concatenate(self.buffer)


def make_env(env_name: str, n_steps=4, render_mode="rgb_array", **kwargs):
    print(f"Creating environment {env_name}")
    env = gym.make(env_name, render_mode=render_mode, **kwargs)
    env = atari_wrappers.AtariWrapper(env, clip_reward=False, noop_max=0)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps=n_steps)
    return env

In [25]:
import re
import os
import pandas as pd

MODELS_DIR = "/content/drive/MyDrive/PUBLIC/Models"

# pattern to extract reward ("_ -14 " etc)
reward_pattern = re.compile(r"best_(-?\d+)", re.IGNORECASE)

# scan all .dat files
dat_files = [f for f in os.listdir(MODELS_DIR) if f.endswith(".dat")]

records = []
for f in dat_files:
    full_path = os.path.join(MODELS_DIR, f)

    game = "Unknown" # Default game name

    # Robustly extract game identifier (e.g., "ALE/Tetris-v5") from filename
    # Filenames are typically "ALE_GameName-vX-best_..."
    game_id_match = re.search(r"(ALE_[a-zA-Z0-9_-]+-v\d+)", f)
    if game_id_match:
        # Convert "ALE_GameName-vX" to "ALE/GameName-vX" to match environment names
        game = game_id_match.group(1).replace("_", "/")

    # extract reward
    reward_match = reward_pattern.search(f)
    reward = int(reward_match.group(1)) if reward_match else None

    # extract timestamp (YYYYMMDD-HHMM)
    time_match = re.search(r"(\d{8}-\d{4})", f)
    timestamp = time_match.group(1) if time_match else None

    records.append({
        "game": game,
        "filename": f,
        "full_path": full_path,
        "reward": reward,
        "timestamp": timestamp,
        "filesize_kb": round(os.path.getsize(full_path) / 1024, 1)
    })

df_all = pd.DataFrame(records)

df_tetris = df_all[df_all["game"] == "ALE/Tetris-v5"].sort_values("timestamp")
df_frogger = df_all[df_all["game"] == "ALE/Frogger-v5"].sort_values("timestamp")

# Handle cases where DataFrames might be empty
latest_tetris = None
best_tetris = None
if not df_tetris.empty:
    latest_tetris = df_tetris.iloc[-1]["full_path"]
    best_tetris = df_tetris.sort_values("reward", ascending=False).iloc[0]["full_path"]

frogger_model_paths = []
if not df_frogger.empty:
    frogger_model_paths = df_frogger["full_path"].tolist()

print("df_all:\n", df_all)
print("\ndf_tetris:\n", df_tetris)
print("\ndf_frogger:\n", df_frogger)
print("\nLatest Tetris Model:", latest_tetris)
print("Best Tetris Model:", best_tetris)
print("Frogger Model Paths:", frogger_model_paths)

  return datetime.utcnow().replace(tzinfo=utc)



df_all:
                game                                           filename  \
0  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_2-20250924-1610-test_eps...   
1  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_3-20250924-1610-test_eps...   
2  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_5-20250924-1610-test_eps...   
3  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_0-20250924-1641-test_eps...   
4  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_5-20250924-1641-test_eps...   
5  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_4-20250924-1700-test_eps...   
6  ALE/Atlantis2-v5  ALE_Atlantis2-v5-best_29-20250924-1700-test_ep...   

                                           full_path  reward      timestamp  \
0  /content/drive/MyDrive/PUBLIC/Models/ALE_Atlan...       2  20250924-1610   
1  /content/drive/MyDrive/PUBLIC/Models/ALE_Atlan...       3  20250924-1610   
2  /content/drive/MyDrive/PUBLIC/Models/ALE_Atlan...       5  20250924-1610   
3  /content/drive/MyDrive/PUBLIC/Models/ALE_Atlan...       0  20250924-1641   
4  

In [26]:
# latest_tetris = df_tetris.iloc[-1]["full_path"]
# latest_tetris

# best_tetris = df_tetris.sort_values("reward", ascending=False).iloc[0]["full_path"]
# best_tetris

# frogger_model_paths = df_frogger["full_path"].tolist()
# frogger_model_paths

# These loops will now correctly use the `latest_tetris`, `best_tetris`, and `frogger_model_paths`
# variables which are now safely initialized to None or empty list if no models are found.
# If `generate_video_from_model` is expected to handle None or empty lists,
# or if it's meant to be called only when models exist, further checks might be needed here.

# For now, I'm commenting out the video generation as `generate_video_from_model` is not defined.
# If you intend to generate videos, please define the function and uncomment the following lines.

# for _, row in df_tetris.iterrows():
#     if latest_tetris is not None: # Only process if Tetris models were found
#         output_mp4 = row["filename"].replace(".dat", ".mp4")
#         print("Creating:", output_mp4)
#         generate_video_from_model(row["full_path"], output_mp4)

# for _, row in df_frogger.iterrows():
#     if frogger_model_paths: # Only process if Frogger models were found
#         output_mp4 = row["filename"].replace(".dat", ".mp4")
#         print("Creating:", output_mp4)
#         generate_video_from_model(row["full_path"], output_mp4)

In [27]:
# Base Configuration
DEFAULT_ENV_NAME = game1_ale
MEAN_REWARD_BOUND = 19

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

SAVE_EPSILON = 0.5  # Only save if at least this much better
EPSILON_DECAY_LAST_FRAME = 150000
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

# Tuple of tensors returned from a sampled minibatch in replay buffer
State = np.ndarray
Action = int
BatchTensors = tt.Tuple[
    torch.ByteTensor,           # current state
    torch.LongTensor,           # actions
    torch.Tensor,               # rewards
    torch.BoolTensor,           # done || trunc
    torch.ByteTensor            # next state
]

In [28]:
# ⚙️ Fast Training Config for Quick Test Run
MEAN_REWARD_BOUND = 5
REPLAY_START_SIZE = 1000
EPSILON_DECAY_LAST_FRAME = 10_000
SYNC_TARGET_FRAMES = 500

# REPLAY_SIZE = 5000  # optional
# BATCH_SIZE = 16     # optional

In [29]:
import os
from pathlib import Path

# Define directories
save_dir_drive = "/content/drive/MyDrive/PUBLIC/Models"
save_dir_local = "saved_models"

# Create both directories if they don't exist
os.makedirs(save_dir_drive, exist_ok=True)
os.makedirs(save_dir_local, exist_ok=True)

# Safe model filename
env_name = DEFAULT_ENV_NAME
safe_env_name = env_name.replace("/", "_")

In [30]:
@dataclass
class Experience:
    state: State
    action: Action
    reward: float
    done_trunc: bool
    new_state: State


class ExperienceBuffer:
    def __init__(self, capacity: int):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience: Experience):
        self.buffer.append(experience)

    def sample(self, batch_size: int) -> tt.List[Experience]:
        indices = np.random.choice(len(self), batch_size, replace=False)
        return [self.buffer[idx] for idx in indices]

In [31]:
class Agent:
    def __init__(self, env: gym.Env, exp_buffer: ExperienceBuffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self.state: tt.Optional[np.ndarray] = None
        self._reset()

    def _reset(self):
        self.state, _ = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net: DQN, device: torch.device,
                  epsilon: float = 0.0) -> tt.Optional[float]:
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_v = torch.as_tensor(self.state).to(device)
            state_v.unsqueeze_(0)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, is_tr, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(
            state=self.state, action=action, reward=float(reward),
            done_trunc=is_done or is_tr, new_state=new_state
        )
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done or is_tr:
            done_reward = self.total_reward
            self._reset()
        return done_reward

In [32]:
def batch_to_tensors(batch: tt.List[Experience], device: torch.device) -> BatchTensors:
    states, actions, rewards, dones, new_state = [], [], [], [], []
    for e in batch:
        states.append(e.state)
        actions.append(e.action)
        rewards.append(e.reward)
        dones.append(e.done_trunc)
        new_state.append(e.new_state)
    states_t = torch.as_tensor(np.asarray(states))
    actions_t = torch.LongTensor(actions)
    rewards_t = torch.FloatTensor(rewards)
    dones_t = torch.BoolTensor(dones)
    new_states_t = torch.as_tensor(np.asarray(new_state))
    return states_t.to(device), actions_t.to(device), rewards_t.to(device), \
           dones_t.to(device),  new_states_t.to(device)

In [33]:
def calc_loss(batch: tt.List[Experience], net: DQN, tgt_net: DQN,
              device: torch.device) -> torch.Tensor:
    states_t, actions_t, rewards_t, dones_t, new_states_t = batch_to_tensors(batch, device)

    state_action_values = net(states_t).gather(
        1, actions_t.unsqueeze(-1)
    ).squeeze(-1)
    with torch.no_grad():
        next_state_values = tgt_net(new_states_t).max(1)[0]
        next_state_values[dones_t] = 0.0
        next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_t
    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [ ]:
model_comment = f"test_epsdec{EPSILON_DECAY_LAST_FRAME}_rs{REPLAY_START_SIZE}_sync{SYNC_TARGET_FRAMES}"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = make_env(env_name)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

start_time = time.time()
while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time  # in seconds
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
        #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or m_reward > best_m_reward + SAVE_EPSILON:
            print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
            timestamp = datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"

            # Save to both paths
            model_path_drive = os.path.join(save_dir_drive, model_filename)
            model_path_local = os.path.join(save_dir_local, model_filename)

            torch.save(net.state_dict(), model_path_drive)
            torch.save(net.state_dict(), model_path_local)

            print(f"💾 Model saved to:\n - Google Drive: {model_path_drive}\n - Local:        {model_path_local}")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward
        if m_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break
    if len(buffer) < REPLAY_START_SIZE:
        continue
    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()
env.close()
writer.close()

Creating environment ALE/Tetris-v5
DQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=5, bias=True)
  )
)


  return datetime.utcnow().replace(tzinfo=utc)



155: done 1 games, reward 0.000, eps 0.98, speed 87.99 f/s, time 0.0 min
💾 Model saved to:
 - Google Drive: /content/drive/MyDrive/PUBLIC/Models/ALE_Tetris-v5-best_0-20251127-0725-test_epsdec10000_rs1000_sync500.dat
 - Local:        saved_models/ALE_Tetris-v5-best_0-20251127-0725-test_epsdec10000_rs1000_sync500.dat


# ----- GAME 2 -----

# GAME 2 Model

In [ ]:
#dqn_model
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )
        size = self.conv(torch.zeros(1, *input_shape)).size()[-1]
        self.fc = nn.Sequential(
            nn.Linear(size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    def forward(self, x: torch.ByteTensor):
        x = x.float() / 255.0
        return self.fc(self.conv(x))

In [ ]:
#wrappers

from gymnasium import spaces
from stable_baselines3.common import atari_wrappers


class ImageToPyTorch(gym.ObservationWrapper):
    """
    ImageToPyTorch: Reorders image dimensions from (H, W, C) to (C, H, W)
    for compatibility with PyTorch convolutional layers.
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        obs = self.observation_space
        assert isinstance(obs, gym.spaces.Box)
        assert len(obs.shape) == 3
        new_shape = (obs.shape[-1], obs.shape[0], obs.shape[1])
        self.observation_space = gym.spaces.Box(
            low=obs.low.min(), high=obs.high.max(),
            shape=new_shape, dtype=obs.dtype)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class BufferWrapper(gym.ObservationWrapper):
    """
    BufferWrapper: Maintains a rolling window of the last `n_steps` frames
    to give the agent a sense of temporal context.
    """
    def __init__(self, env, n_steps):
        super(BufferWrapper, self).__init__(env)
        obs = env.observation_space
        assert isinstance(obs, spaces.Box)
        new_obs = gym.spaces.Box(
            obs.low.repeat(n_steps, axis=0), obs.high.repeat(n_steps, axis=0),
            dtype=obs.dtype)
        self.observation_space = new_obs
        self.buffer = collections.deque(maxlen=n_steps)

    def reset(self, *, seed: tt.Optional[int] = None, options: tt.Optional[dict[str, tt.Any]] = None):
        for _ in range(self.buffer.maxlen):
            self.buffer.append(np.zeros_like(self.env.observation_space.low))
        obs, extra = self.env.reset()
        return self.observation(obs), extra

    def observation(self, observation: np.ndarray) -> np.ndarray:
        self.buffer.append(observation)
        return np.concatenate(self.buffer)


def make_env(env_name: str, n_steps=4, render_mode="rgb_array", **kwargs):
    print(f"Creating environment {env_name}")
    env = gym.make(env_name, render_mode=render_mode, **kwargs)
    env = atari_wrappers.AtariWrapper(env, clip_reward=False, noop_max=0)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, n_steps=n_steps)
    return env

In [ ]:
MODELS_DIR = "/content/drive/MyDrive/PUBLIC/Models"

# pattern to extract reward ("_ -14 " etc)
reward_pattern = re.compile(r"best_(-?\d+)", re.IGNORECASE)

# scan all .dat files
dat_files = [f for f in os.listdir(MODELS_DIR) if f.endswith(".dat")]

records = []
for f in dat_files:
    full_path = os.path.join(MODELS_DIR, f)

    # detect game name (Tetris-v5, Frogger-v5, Pong-v5, etc.)
    game = None
    if "Tetris" in f or "tetris" in f:
        game = "Tetris-v5"
    elif "Frogger" in f or "frogger" in f:
        game = "Frogger-v5"
    elif "Pong" in f or "pong" in f:
        game = "Pong-v5"
    else:
        game = "Unknown"

    # extract reward
    reward_match = reward_pattern.search(f)
    reward = int(reward_match.group(1)) if reward_match else None

    # extract timestamp (YYYYMMDD-HHMM)
    time_match = re.search(r"(\d{8}-\d{4})", f)
    timestamp = time_match.group(1) if time_match else None

    records.append({
        "game": game,
        "filename": f,
        "full_path": full_path,
        "reward": reward,
        "timestamp": timestamp,
        "filesize_kb": round(os.path.getsize(full_path) / 1024, 1)
    })

df_all = pd.DataFrame(records)
df_all

df_tetris = df_all[df_all["game"] == "Tetris-v5"].sort_values("timestamp")
df_tetris

df_frogger = df_all[df_all["game"] == "Frogger-v5"].sort_values("timestamp")
df_frogger


latest_tetris = df_tetris.iloc[-1]["full_path"]
latest_tetris

# Tetris, highest reward:
best_tetris = df_tetris.sort_values("reward", ascending=False).iloc[0]["full_path"]
best_tetris

# frogger highest reward:
frogger_model_paths = df_frogger["full_path"].tolist()
frogger_model_paths

In [ ]:
latest_tetris = df_tetris.iloc[-1]["full_path"]
latest_tetris

best_tetris = df_tetris.sort_values("reward", ascending=False).iloc[0]["full_path"]
best_tetris

frogger_model_paths = df_frogger["full_path"].tolist()
frogger_model_paths

for _, row in df_tetris.iterrows():
    output_mp4 = row["filename"].replace(".dat", ".mp4")
    print("Creating:", output_mp4)
    generate_video_from_model(row["full_path"], output_mp4)

for _, row in df_frogger.iterrows():
    output_mp4 = row["filename"].replace(".dat", ".mp4")
    print("Creating:", output_mp4)
    generate_video_from_model(row["full_path"], output_mp4)


In [ ]:
# Base Configuration
DEFAULT_ENV_NAME = game2_ale
MEAN_REWARD_BOUND = 19

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

SAVE_EPSILON = 0.5  # Only save if at least this much better
EPSILON_DECAY_LAST_FRAME = 150000
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

# Tuple of tensors returned from a sampled minibatch in replay buffer
State = np.ndarray
Action = int
BatchTensors = tt.Tuple[
    torch.ByteTensor,           # current state
    torch.LongTensor,           # actions
    torch.Tensor,               # rewards
    torch.BoolTensor,           # done || trunc
    torch.ByteTensor            # next state
]

In [ ]:
# ⚙️ Fast Training Config for Quick Test Run
MEAN_REWARD_BOUND = 5
REPLAY_START_SIZE = 1000
EPSILON_DECAY_LAST_FRAME = 10_000
SYNC_TARGET_FRAMES = 500

# REPLAY_SIZE = 5000  # optional
# BATCH_SIZE = 16     # optional

In [ ]:
import os
from pathlib import Path

# Define directories
save_dir_drive = "/content/drive/MyDrive/PUBLIC/Models"
save_dir_local = "saved_models"

# Create both directories if they don't exist
os.makedirs(save_dir_drive, exist_ok=True)
os.makedirs(save_dir_local, exist_ok=True)

# Safe model filename
env_name = DEFAULT_ENV_NAME
safe_env_name = env_name.replace("/", "_")

In [ ]:
@dataclass
class Experience:
    state: State
    action: Action
    reward: float
    done_trunc: bool
    new_state: State


class ExperienceBuffer:
    def __init__(self, capacity: int):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience: Experience):
        self.buffer.append(experience)

    def sample(self, batch_size: int) -> tt.List[Experience]:
        indices = np.random.choice(len(self), batch_size, replace=False)
        return [self.buffer[idx] for idx in indices]

In [ ]:
class Agent:
    def __init__(self, env: gym.Env, exp_buffer: ExperienceBuffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self.state: tt.Optional[np.ndarray] = None
        self._reset()

    def _reset(self):
        self.state, _ = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net: DQN, device: torch.device,
                  epsilon: float = 0.0) -> tt.Optional[float]:
        done_reward = None

        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_v = torch.as_tensor(self.state).to(device)
            state_v.unsqueeze_(0)
            q_vals_v = net(state_v)
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        # do step in the environment
        new_state, reward, is_done, is_tr, _ = self.env.step(action)
        self.total_reward += reward

        exp = Experience(
            state=self.state, action=action, reward=float(reward),
            done_trunc=is_done or is_tr, new_state=new_state
        )
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done or is_tr:
            done_reward = self.total_reward
            self._reset()
        return done_reward

In [ ]:
def batch_to_tensors(batch: tt.List[Experience], device: torch.device) -> BatchTensors:
    states, actions, rewards, dones, new_state = [], [], [], [], []
    for e in batch:
        states.append(e.state)
        actions.append(e.action)
        rewards.append(e.reward)
        dones.append(e.done_trunc)
        new_state.append(e.new_state)
    states_t = torch.as_tensor(np.asarray(states))
    actions_t = torch.LongTensor(actions)
    rewards_t = torch.FloatTensor(rewards)
    dones_t = torch.BoolTensor(dones)
    new_states_t = torch.as_tensor(np.asarray(new_state))
    return states_t.to(device), actions_t.to(device), rewards_t.to(device), \
           dones_t.to(device),  new_states_t.to(device)

In [ ]:
def calc_loss(batch: tt.List[Experience], net: DQN, tgt_net: DQN,
              device: torch.device) -> torch.Tensor:
    states_t, actions_t, rewards_t, dones_t, new_states_t = batch_to_tensors(batch, device)

    state_action_values = net(states_t).gather(
        1, actions_t.unsqueeze(-1)
    ).squeeze(-1)
    with torch.no_grad():
        next_state_values = tgt_net(new_states_t).max(1)[0]
        next_state_values[dones_t] = 0.0
        next_state_values = next_state_values.detach()

    expected_state_action_values = next_state_values * GAMMA + rewards_t
    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [ ]:
model_comment = f"test_epsdec{EPSILON_DECAY_LAST_FRAME}_rs{REPLAY_START_SIZE}_sync{SYNC_TARGET_FRAMES}"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = make_env(env_name)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

start_time = time.time()
while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time  # in seconds
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
        #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or m_reward > best_m_reward + SAVE_EPSILON:
            print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
            timestamp = datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"

            # Save to both paths
            model_path_drive = os.path.join(save_dir_drive, model_filename)
            model_path_local = os.path.join(save_dir_local, model_filename)

            torch.save(net.state_dict(), model_path_drive)
            torch.save(net.state_dict(), model_path_local)

            print(f"💾 Model saved to:\n - Google Drive: {model_path_drive}\n - Local:        {model_path_local}")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward
        if m_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break
    if len(buffer) < REPLAY_START_SIZE:
        continue
    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()
env.close()
writer.close()

In [ ]:
MODELS_DIR = "/content/drive/MyDrive/PUBLIC/Models"

# pattern to extract reward ("_ -14 " etc)
reward_pattern = re.compile(r"best_(-?\d+)", re.IGNORECASE)

# scan all .dat files
dat_files = [f for f in os.listdir(MODELS_DIR) if f.endswith(".dat")]

records = []
for f in dat_files:
    full_path = os.path.join(MODELS_DIR, f)

    # detect game name (Tetris-v5, Frogger-v5, Pong-v5, etc.)
    game = None
    if "Tetris" in f or "tetris" in f:
        game = "Tetris-v5"
    elif "Frogger" in f or "frogger" in f:
        game = "Frogger-v5"
    elif "Pong" in f or "pong" in f:
        game = "Pong-v5"
    else:
        game = "Unknown"

    # extract reward
    reward_match = reward_pattern.search(f)
    reward = int(reward_match.group(1)) if reward_match else None

    # extract timestamp (YYYYMMDD-HHMM)
    time_match = re.search(r"(\d{8}-\d{4})", f)
    timestamp = time_match.group(1) if time_match else None

    records.append({
        "game": game,
        "filename": f,
        "full_path": full_path,
        "reward": reward,
        "timestamp": timestamp,
        "filesize_kb": round(os.path.getsize(full_path) / 1024, 1)
    })

df_all = pd.DataFrame(records)
df_all

df_tetris = df_all[df_all["game"] == "Tetris-v5"].sort_values("timestamp")
df_tetris

df_frogger = df_all[df_all["game"] == "Frogger-v5"].sort_values("timestamp")
df_frogger


latest_tetris = df_tetris.iloc[-1]["full_path"]
latest_tetris

# Tetris, highest reward:
best_tetris = df_tetris.sort_values("reward", ascending=False).iloc[0]["full_path"]
best_tetris

# frogger highest reward:
frogger_model_paths = df_frogger["full_path"].tolist()
frogger_model_paths

In [ ]:
latest_tetris = df_tetris.iloc[-1]["full_path"]
latest_tetris

best_tetris = df_tetris.sort_values("reward", ascending=False).iloc[0]["full_path"]
best_tetris

frogger_model_paths = df_frogger["full_path"].tolist()
frogger_model_paths

for _, row in df_tetris.iterrows():
    output_mp4 = row["filename"].replace(".dat", ".mp4")
    print("Creating:", output_mp4)
    generate_video_from_model(row["full_path"], output_mp4)

for _, row in df_frogger.iterrows():
    output_mp4 = row["filename"].replace(".dat", ".mp4")
    print("Creating:", output_mp4)
    generate_video_from_model(row["full_path"], output_mp4)


# Task
Train DQN models for both "ALE/Tetris-v5" and "ALE/Frogger-v5" environments, using a fast training configuration, and save the trained models to Google Drive and a local directory.

## Prepare for Tetris Training

### Subtask:
Set the default environment name to 'ALE/Tetris-v5' and load the training parameters for the Tetris model. This includes the fast training configuration for a quick test run.


The instructions to set the `DEFAULT_ENV_NAME` to `'ALE/Tetris-v5'` and configure the fast training parameters are already handled by the existing code cells.

- `DEFAULT_ENV_NAME = game1_ale` is set in cell `GvXPdjPCBxOd`.
- The fast training parameters (`MEAN_REWARD_BOUND = 5`, `REPLAY_START_SIZE = 1000`, `EPSILON_DECAY_LAST_FRAME = 10_000`, and `SYNC_TARGET_FRAMES = 500`) are defined and override the base configuration in cell `3FHvHNMrR9Sk`.

No further action is needed for this step as the current notebook setup already implements these configurations.

## Train Tetris Model

### Subtask:
Execute the training loop for the Tetris model using the defined DQN architecture, replay buffer, and agent. The model will save progress to Google Drive and a local directory.


**Reasoning**:
The subtask requires executing the training loop for the Tetris model. The cell `Q8gj-5woCXEB` contains the complete training loop logic, including environment setup, model initialization, agent interaction, loss calculation, and model saving.



In [ ]:
model_comment = f"test_epsdec{EPSILON_DECAY_LAST_FRAME}_rs{REPLAY_START_SIZE}_sync{SYNC_TARGET_FRAMES}"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = make_env(env_name)
net = DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = DQN(env.observation_space.shape, env.action_space.n).to(device)
writer = SummaryWriter(comment=f"-{env_name}-{model_comment}")
print(net)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_m_reward = None

start_time = time.time()
while True:
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

    reward = agent.play_step(net, device, epsilon)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx - ts_frame) / (time.time() - ts)
        elapsed = time.time() - start_time  # in seconds
        ts_frame = frame_idx
        ts = time.time()
        m_reward = np.mean(total_rewards[-100:])
        #  print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
        #      f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", m_reward, frame_idx)
        writer.add_scalar("reward", reward, frame_idx)
        if best_m_reward is None or m_reward > best_m_reward + SAVE_EPSILON:
            print(f"{frame_idx}: done {len(total_rewards)} games, reward {m_reward:.3f}, "
                f"eps {epsilon:.2f}, speed {speed:.2f} f/s, time {elapsed/60:.1f} min")
            timestamp = datetime.now().strftime("%Y%m%d-%H%M")
            model_filename = f"{safe_env_name}-best_{int(m_reward)}-{timestamp}-{model_comment}.dat"

            # Save to both paths
            model_path_drive = os.path.join(save_dir_drive, model_filename)
            model_path_local = os.path.join(save_dir_local, model_filename)

            torch.save(net.state_dict(), model_path_drive)
            torch.save(net.state_dict(), model_path_local)

            print(f"💾 Model saved to:\n - Google Drive: {model_path_drive}\n - Local:        {model_path_local}")
            if best_m_reward is not None:
                print(f"Best reward updated {best_m_reward:.3f} -> {m_reward:.3f}")
            best_m_reward = m_reward
        if m_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break
    if len(buffer) < REPLAY_START_SIZE:
        continue
    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device)
    loss_t.backward()
    optimizer.step()
env.close()
writer.close()

**Reasoning**:
The `NameError` indicates that configuration variables like `EPSILON_DECAY_LAST_FRAME` were not defined or loaded into the kernel before the training loop was executed. To fix this, I need to re-run the cells that define the base and fast training configurations, as well as the directory setup, to ensure all necessary global variables are available.



In [ ]:
DEFAULT_ENV_NAME = game1_ale
MEAN_REWARD_BOUND = 19

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000
REPLAY_START_SIZE = 10000

SAVE_EPSILON = 0.5  # Only save if at least this much better
EPSILON_DECAY_LAST_FRAME = 150000
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

# Tuple of tensors returned from a sampled minibatch in replay buffer
State = np.ndarray
Action = int
BatchTensors = tt.Tuple[
    torch.ByteTensor,           # current state
    torch.LongTensor,           # actions
    torch.Tensor,               # rewards
    torch.BoolTensor,           # done || trunc
    torch.ByteTensor            # next state
]